# Machine Learning Models

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from utils.evaluation import get_r, print_metrics

/home/rinjac/anaconda3/envs/deep_nlp/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/rinjac/anaconda3/envs/deep_nlp/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


Load train and test datasets.

In [5]:
X_train = pd.read_csv("../features/weebit_train_with_features.csv", index_col=0)
X_test = pd.read_csv("../features/weebit_test_with_features.csv", index_col=0)

# get Y
y_train = X_train["Level"]
y_test = X_test["Level"]

# remove Y and Text columns 
X_train.drop(columns=['Text', 'Level'], inplace=True)
X_test.drop(columns=['Text', 'Level'], inplace=True)

## 1. Linear Regression

In [33]:
from sklearn.linear_model import Ridge

In [36]:
lin_reg = Ridge(alpha=0.1)

lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_test)

print_metrics(y_test, y_pred)

R^2 = 0.5189337005930125
R = 0.7203705300697777
Pearson's correlation coef: 0.7252042670577996


## 2. Random Forest Classifier

In [16]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
rfc = RandomForestClassifier(max_depth=20, n_estimators=100)

In [27]:
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [43]:
print_metrics(y_test, y_pred, classification=True)

Confusion matrix:
[[ 80  31   5   4   2]
 [ 17 112  18   5   6]
 [  3  27 114   3  13]
 [  5   7   6  82  28]
 [  2   7  10  27 114]]
Accuracy: 0.6895604395604396
F1(micro)0.6895604395604396
F1(macro)0.6893224224903831
R^2 = 0.5811758475813705
R = 0.7623489014758076
Pearson's correlation coef: 0.7884321525669032


## 3. XGBoost Classifier

In [6]:
from xgboost import XGBClassifier

In [40]:
xgboost = XGBClassifier(max_depth=30, n_estimators=200, learning_rate=0.1)

In [41]:
xgboost.fit(X_train, y_train)
y_pred = xgboost.predict(X_test)

In [42]:
print_metrics(y_test, y_pred)

R^2 = 0.5811758475813705
R = 0.7623489014758076
Pearson's correlation coef: 0.7884321525669032


## 4. Support Vector Classifier

In [6]:
from sklearn.svm import SVC
from utils.hyperparameter_optimization import find_best_C_for_SVC

Perform hyperparameter optimization to find best C.

In [7]:
kernel = 'linear'
Cs = [0.1, 0.3, 0.5, 0.7, 1.0, 1.5, 2.0, 5.0, 10.0, 50.0]
bestC = find_best_C_for_SVC(X_train, y_train, kernel, Cs)

Train and evaluate the model.

In [9]:
svc = SVC(kernel=kernel, C=bestC)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
    
print_metrics(y_test, y_pred, classification=True)

R^2 = 0.5634590898648464
R = 0.750639120926192
Pearson's correlation coef: 0.7810546559623373


## 5. Simple Feedforward Network

In [101]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical

In [159]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [160]:
y_train_cat = to_categorical(y_train, num_classes=5)
y_test_cat = to_categorical(y_test, num_classes=5)

In [161]:
model.fit(X_train, y_train_cat,
          epochs=150,
          batch_size=64,
          verbose=1)

Epoch 1/150
2911/2911 [==============================] - 2s 637us/step - loss: 1.9691 - acc: 0.2219
Epoch 2/150
2911/2911 [==============================] - 0s 62us/step - loss: 1.6057 - acc: 0.2360
Epoch 3/150
2911/2911 [==============================] - 0s 70us/step - loss: 1.5711 - acc: 0.2597
Epoch 4/150
2911/2911 [==============================] - 0s 73us/step - loss: 1.5581 - acc: 0.2772
Epoch 5/150
2911/2911 [==============================] - 0s 63us/step - loss: 1.5311 - acc: 0.2906
Epoch 6/150
2911/2911 [==============================] - 0s 65us/step - loss: 1.5187 - acc: 0.3057
Epoch 7/150
2911/2911 [==============================] - 0s 69us/step - loss: 1.5006 - acc: 0.3147
Epoch 8/150
2911/2911 [==============================] - 0s 70us/step - loss: 1.4788 - acc: 0.3136
Epoch 9/150
2911/2911 [==============================] - 0s 91us/step - loss: 1.4741 - acc: 0.3473
Epoch 10/150
2911/2911 [==============================] - 0s 93us/step - loss: 1.4544 - acc: 0.3456
Epoch 11

In [162]:
y_pred_cat = model.predict(X_test)

In [163]:
y_pred = np.argmax(y_pred_cat, axis=1)

In [164]:
print_metrics(y_test, y_pred, classification=True)

Confusion matrix:
[[ 75  37   3   6   1]
 [ 20 115  14   4   5]
 [  2  33  99   4  22]
 [  2  10   6  79  31]
 [  3   2  13  27 115]]
Accuracy: 0.6634615384615384
F1(micro)0.6634615384615384
F1(macro)0.6628872300898104
R^2 = 0.5840105288160145
R = 0.7642058157433863
Pearson's correlation coef: 0.7924956207300548
